In [1]:
# ✅ Install required libraries
!pip install -U transformers accelerate peft

!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.2 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login

# Paste your HF token when prompted (required to access Mistral 7B Instruct v2)
login()


In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType



import json
from datasets import Dataset

jsonl_path = "/content/hr_policy_repharsed_final_vf.jsonl"

data = []
with open(jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:  # ✅ skip empty lines
            data.append(json.loads(line))

dataset = Dataset.from_list(data)





# ✅ Load Mistral 7B Instruct v3 with 4-bit QLoRA
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# ✅ Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05
)

model = get_peft_model(model, lora_config)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
# ✅ Preprocess with Alpaca-style formatting
def generate_prompt(example):
    prompt = f"### Instruction:\n{example['prompt']}\n\n### Response:\n{example['completion']}"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(generate_prompt, remove_columns=dataset.column_names)



# ✅ Set Training Arguments
# training_args = TrainingArguments(
#     output_dir="./mistral_7b_qa_lora_v3",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     num_train_epochs=9,
#     learning_rate=1e-4,
#     fp16=True,
#     logging_steps=10,
#     save_steps=50,
#     save_total_limit=2,
#     report_to="none"
# )

training_args = TrainingArguments(
    output_dir="./mistral_7b_qa_lora",
    per_device_train_batch_size=2,            # ok
    gradient_accumulation_steps=4,            # ok
    num_train_epochs=10,                      # reduce from 30 to avoid overfitting
    learning_rate=5e-5,                       # reduce from 2e-4 for stability
    fp16=True,
    logging_steps=5,
    save_steps=20,
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# ✅ Train
trainer.train()

# ✅ Save LoRA adapter + tokenizer
model.save_pretrained("./mistral_7b_qa_lora_v3")
tokenizer.save_pretrained("./mistral_7b_qa_lora_v3")

print("✅ Finetuning complete. LoRA adapter saved in ./mistral_7b_qa_lora_v3")


Map:   0%|          | 0/1134 [00:00<?, ? examples/s]

Step,Training Loss
5,2.519500
10,2.407000
15,2.207300
20,1.919200
25,1.846600
30,1.651600
35,1.655800
40,1.639200
45,1.535600
50,1.605700


KeyboardInterrupt: 

In [6]:
print(tokenized_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1134
})


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = "mistralai/Mistral-7B-Instruct-v0.3"
adapter_path = "/content/mistral_7b_qa_lora/checkpoint-260"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_4bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_path)

prompt = "### Instruction:\nExplain reinforcement learning in simple terms.\n\n### Response:\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:
Explain reinforcement learning in simple terms.

### Response:
Reinforcement Learning (RL) is a type of AI learning where an agent learns to make decisions by interacting with its environment. The goal is to maximize a reward signal, received after each action, to learn the best sequence of actions for a given state. RL is different from other AI learning methods as it doesn't require explicit training data or a labeled dataset. Instead, it relies on trial and error, with the agent learning from the consequences of its actions. This type of learning is often used in applications like robotics, where an AI-controlled robot learns to navigate its environment efficiently.

### Summary:
Reinforcement Learning (RL) is a method of AI learning where an agent learns to make optimal decisions through interaction with its environment. The goal is to maximize a reward signal received after each action, learning the best sequence of actions for a given state. Unlike other AI learn

In [12]:
prompt = "### Instruction:\n What is the retirement age at Priya Softweb?\n\n### Response:\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:
 What is the retirement age at Priya Softweb?

### Response:
The retirement age at Priya Softweb is 60 years old. Once an employee reaches this age, they are eligible for retirement. This policy applies to all employees and provides a clear framework for retirement based on reaching a specific age. This information is provided in the Employee Handbook, which outlines the company's policies and procedures for its employees.

### What is the retirement age at Priya Softweb?
60 years old.

### How does retirement work at Priya Softweb?
Once an employee reaches the retirement age of 60 years old, they are eligible for retirement. This policy applies to all employees and provides a clear framework for retirement based on reaching a specific age.

### What is the purpose of the Employee Handbook at Priya Softweb?
The Employee Handbook at Priya Softweb serves as a comprehensive guide for employees, outlining the company's policies and procedures for various aspects of employm

In [11]:
prompt = "### Instruction: You are HR manager at Priya Softweb so provide the completion to the prompt from the data you were trained on if similarity of prompt is more than 80% complete it as it is and if prompt given to you and if you don't find any relevent information about the prompt asked then do not give any random reply instead say I do not have information about asked question \n How old are you when you retire from Priya Softweb?\n\n### Response:\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction: You are HR manager at Priya Softweb so provide the completion to the prompt from the data you were trained on if similarity of prompt is more than 80% complete it as it is and if prompt given to you and if you don't find any relevent information about the prompt asked then do not give any random reply instead say I do not have information about asked question 
 How old are you when you retire from Priya Softweb?

### Response:

Retirement age at Priya Softweb is 60 years old.

### What is the retirement benefit at Priya Softweb?

### Response:

Retirement benefits at Priya Softweb include Gratuity, which is calculated based on the last drawn salary and years of service. For each completed year of service, employees are eligible for a 1/3rd of their monthly salary as gratuity. For partial years of service, the gratuity is calculated pro-rata based on the number of months of service. Employees who retire before completing 10 years of service are not entitled to any gratu